Details of the dataset with ML models can be found in: https://www.kaggle.com/prasadperera/the-boston-housing-dataset

In [1]:
import jovian
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import numpy as np

In [18]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 100 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [3]:
allData = np.genfromtxt('housing.csv',delimiter='')
allData = np.float32(allData)

num_inputdata, num_inputparam = allData.shape
num_inputparam -= 1

print("No. of input data points = ", num_inputdata)
print("No. of input parameters = ", num_inputparam)

No. of input data points =  506
No. of input parameters =  13


In [4]:
allData.dtype

dtype('float32')

In [5]:
# Input (temp, rainfall, humidity)
inputs = np.array(allData[:,0:13])
print("Input array: ", inputs)
# Targets (apples, oranges)
targets = np.array([allData[:,13]]).T
print("Target array: ", targets)


Input array:  [[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
Target array:  [[24. ]
 [21.6]
 [34.7]
 [33.4]
 [36.2]
 [28.7]
 [22.9]
 [27.1]
 [16.5]
 [18.9]
 [15. ]
 [18.9]
 [21.7]
 [20.4]
 [18.2]
 [19.9]
 [23.1]
 [17.5]
 [20.2]
 [18.2]
 [13.6]
 [19.6]
 [15.2]
 [14.5]
 [15.6]
 [13.9]
 [16.6]
 [14.8]
 [18.4]
 [21. ]
 [12.7]
 [14.5]
 [13.2]
 [13.1]
 [13.5]
 [18.9]
 [20. ]
 [21. ]
 [24.7]
 [30.8]
 [34.9]
 [26.6]
 [25.3]
 [24.7]
 [21.2]
 [19.3]
 [20. ]
 [16.6]
 [14.4]
 [19.4]
 [19.7]
 [20.5]
 [25. ]
 [23.4]
 [18.9]
 [35.4]
 [24.7]
 [31.6]
 [23.3]
 [19.6]
 [18.7]
 [16. ]
 [22.2]
 [25. ]
 [33. ]
 [23.5]

In [6]:
# Convert numpy arrays to PyTorch tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print("Input tensor: ", inputs)
print("Target tensor: ", targets)

Input tensor:  tensor([[6.3200e-03, 1.8000e+01, 2.3100e+00,  ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00,  ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00,  ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01,  ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01,  ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01,  ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]])
Target tensor:  tensor([[24.0000],
        [21.6000],
        [34.7000],
        [33.4000],
        [36.2000],
        [28.7000],
        [22.9000],
        [27.1000],
        [16.5000],
        [18.9000],
        [15.0000],
        [18.9000],
        [21.7000],
        [20.4000],
        [18.2000],
        [19.9000],
        [23.1000],
        [17.5000],
        [20.2000],
        [18.

In [7]:
norm_inputs = F.normalize(inputs, p=2.0, dim=1, eps=1e-12, out=None)

In [8]:
# Define dataset
train_ds = TensorDataset(norm_inputs, targets)

# Define data loader
batch_size = 22
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[2.6529e-05, 7.1368e-02, 3.0994e-03, 0.0000e+00, 9.0128e-04, 1.4765e-02,
         1.0053e-01, 1.4351e-02, 2.0391e-03, 5.7910e-01, 3.1606e-02, 8.0491e-01,
         1.1195e-02],
        [3.6806e-04, 0.0000e+00, 1.4958e-02, 0.0000e+00, 9.9924e-04, 1.2923e-02,
         1.1006e-01, 9.2027e-03, 1.0134e-02, 5.8171e-01, 3.9726e-02, 8.0446e-01,
         1.3924e-02],
        [6.4554e-05, 1.5459e-01, 3.8039e-03, 0.0000e+00, 7.7764e-04, 1.1547e-02,
         7.1955e-02, 1.1749e-02, 3.7477e-03, 6.5210e-01, 2.7545e-02, 7.3786e-01,
         1.3923e-02],
        [2.6898e-04, 0.0000e+00, 1.5149e-02, 0.0000e+00, 9.8218e-04, 1.3306e-02,
         8.7695e-02, 1.2542e-02, 6.5771e-03, 5.1082e-01, 3.9243e-02, 8.5369e-01,
         2.0937e-02],
        [7.4897e-05, 5.1173e-02, 9.9481e-03, 0.0000e+00, 8.7199e-04, 1.2900e-02,
         6.5911e-02, 1.1055e-02, 8.1877e-03, 5.7519e-01, 3.8892e-02, 8.1243e-01,
         1.3755e-02],
        [8.4353e-03, 0.0000e+00, 2.4281e-02, 0.0000e+00, 9.9269e-04, 8.5063e-03,

In [9]:
# Define model
model = nn.Linear(num_inputparam, 1)          #nn.Linear(num_input_param, num_target_param)

# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0087, -0.1344,  0.0029, -0.1647, -0.2538, -0.2741, -0.0843, -0.2712,
          -0.1353,  0.0533,  0.1432, -0.1405, -0.1945]], requires_grad=True),
 Parameter containing:
 tensor([-0.2620], requires_grad=True)]

In [10]:
# Generate predictions
preds = model(norm_inputs)
preds

tensor([[-0.3618],
        [-0.3727],
        [-0.3680],
        [-0.3706],
        [-0.3732],
        [-0.3732],
        [-0.3614],
        [-0.3683],
        [-0.3714],
        [-0.3652],
        [-0.3683],
        [-0.3644],
        [-0.3578],
        [-0.3544],
        [-0.3561],
        [-0.3534],
        [-0.3473],
        [-0.3581],
        [-0.3322],
        [-0.3553],
        [-0.3609],
        [-0.3594],
        [-0.3622],
        [-0.3633],
        [-0.3614],
        [-0.3461],
        [-0.3581],
        [-0.3478],
        [-0.3596],
        [-0.3573],
        [-0.3591],
        [-0.3586],
        [-0.3353],
        [-0.3571],
        [-0.3380],
        [-0.3622],
        [-0.3593],
        [-0.3581],
        [-0.3564],
        [-0.3799],
        [-0.3781],
        [-0.3602],
        [-0.3607],
        [-0.3627],
        [-0.3688],
        [-0.3682],
        [-0.3698],
        [-0.3797],
        [-0.3859],
        [-0.3757],
        [-0.3767],
        [-0.3777],
        [-0.

In [11]:
# Define loss function
loss_fn = F.mse_loss

loss = loss_fn(model(norm_inputs), targets)
print(loss)

tensor(607.8229, grad_fn=<MseLossBackward0>)


In [27]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-7)

In [28]:
# Define number of terations
num_epoch = 10000

In [29]:
fit(num_epoch, model, loss_fn, opt, train_dl)

Epoch [100/10000], Loss: 288.9254
Epoch [200/10000], Loss: 147.1155
Epoch [300/10000], Loss: 127.3507
Epoch [400/10000], Loss: 64.1131
Epoch [500/10000], Loss: 189.0746
Epoch [600/10000], Loss: 158.7723
Epoch [700/10000], Loss: 135.1568
Epoch [800/10000], Loss: 127.7908
Epoch [900/10000], Loss: 111.0645
Epoch [1000/10000], Loss: 83.7009
Epoch [1100/10000], Loss: 175.4854
Epoch [1200/10000], Loss: 181.8068
Epoch [1300/10000], Loss: 136.0263
Epoch [1400/10000], Loss: 118.9704
Epoch [1500/10000], Loss: 90.5398
Epoch [1600/10000], Loss: 89.8956
Epoch [1700/10000], Loss: 245.0852
Epoch [1800/10000], Loss: 204.1262
Epoch [1900/10000], Loss: 110.6879
Epoch [2000/10000], Loss: 109.2860
Epoch [2100/10000], Loss: 124.7536
Epoch [2200/10000], Loss: 173.6064
Epoch [2300/10000], Loss: 238.7163
Epoch [2400/10000], Loss: 198.2694
Epoch [2500/10000], Loss: 163.3807
Epoch [2600/10000], Loss: 169.3191
Epoch [2700/10000], Loss: 96.6592
Epoch [2800/10000], Loss: 120.3806
Epoch [2900/10000], Loss: 171.9427

In [30]:
# Generate predictions
preds = model(norm_inputs)
preds

tensor([[14.9265],
        [14.8106],
        [14.8290],
        [14.7670],
        [14.7622],
        [14.7669],
        [14.9416],
        [14.9150],
        [14.9064],
        [14.9317],
        [14.9181],
        [14.9309],
        [14.9420],
        [14.9411],
        [14.9384],
        [14.9429],
        [14.9410],
        [14.9352],
        [14.9495],
        [14.9423],
        [14.9188],
        [14.9256],
        [14.9175],
        [14.9087],
        [14.9179],
        [14.9293],
        [14.9317],
        [14.9260],
        [14.9226],
        [14.9346],
        [14.9270],
        [14.9208],
        [14.8002],
        [14.9296],
        [14.8207],
        [14.9001],
        [14.9223],
        [14.9016],
        [14.8991],
        [14.7650],
        [14.7587],
        [14.7783],
        [14.7879],
        [14.7678],
        [14.8077],
        [14.7919],
        [14.7894],
        [14.7781],
        [14.7484],
        [14.7910],
        [14.8216],
        [14.8244],
        [14.

In [24]:
targets

tensor([[24.0000],
        [21.6000],
        [34.7000],
        [33.4000],
        [36.2000],
        [28.7000],
        [22.9000],
        [27.1000],
        [16.5000],
        [18.9000],
        [15.0000],
        [18.9000],
        [21.7000],
        [20.4000],
        [18.2000],
        [19.9000],
        [23.1000],
        [17.5000],
        [20.2000],
        [18.2000],
        [13.6000],
        [19.6000],
        [15.2000],
        [14.5000],
        [15.6000],
        [13.9000],
        [16.6000],
        [14.8000],
        [18.4000],
        [21.0000],
        [12.7000],
        [14.5000],
        [13.2000],
        [13.1000],
        [13.5000],
        [18.9000],
        [20.0000],
        [21.0000],
        [24.7000],
        [30.8000],
        [34.9000],
        [26.6000],
        [25.3000],
        [24.7000],
        [21.2000],
        [19.3000],
        [20.0000],
        [16.6000],
        [14.4000],
        [19.4000],
        [19.7000],
        [20.5000],
        [25.